In [ ]:
import os
import numpy as np
from numpy.linalg import svd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path as pth
from pyslingshot import Slingshot
import random
import anndata as ad
from sklearn.decomposition import PCA

/Users/canderson/miniconda3/envs/traj-alignment/lib/python3.12/site-packages/pyslingshot/slingshot.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Set Working Directory

In [ ]:
os.chdir(pth.home() / 'traj-alignment')

### Load Data